# How to build a custom environment with custom observations, rewards and constraints.
In this notebook, we show how to build a custom environment for maize management with custom observations, rewards and constraints.
We start by importing the required packages.

In [1]:
import numpy as np
from cyclesgym.utils.paths import PROJECT_PATH
from stable_baselines3.common.vec_env import SubprocVecEnv, VecNormalize
from stable_baselines3.common.vec_env import VecMonitor

We first define a new observation space. A simple useful observation for maize fertilization is the cumulative amount of fertilizer used so far in the season.
This observation is already available in 'observers'.

In [2]:
from cyclesgym.envs.observers import NToDateObserver

A new observation to combine with the last one, is a trailing window of the fertilization events in the last month. Given that the standard corn evironment has a step size of 1 week,
this trailing window can be a 4 dimensional vector.

In [3]:
from cyclesgym.envs.observers import Observer
from datetime import datetime

class FertilizationTrailingWindowObserver(Observer):

    def __init__(self,
                 end_year: int,
                 Nobs=4):
        super(FertilizationTrailingWindowObserver, self).__init__(end_year)
        self.Nobs = Nobs
        self.lower_bound = np.full((self.Nobs,), -np.inf)
        self.upper_bound = np.full((self.Nobs,), np.inf)
        self.reset()

    def compute_obs(self,
                    date: datetime.date,
                    N: int):

        self.window.append(N)
        self.window = self.window[-self.Nobs:]
        return np.array(self.window)

    def reset(self):
        self.window = [0]*self.Nobs
        self.obs_names = ['WINDOW_'+str(i) for i in range(self.Nobs)]

We can now compose the trailing window observation and the cumulative nitrogen observation as

In [4]:
from cyclesgym.envs.observers import compound_observer
trailing_and_cumulative = lambda end_year: compound_observer([NToDateObserver(end_year), FertilizationTrailingWindowObserver(end_year)])

We can now build a new action space that uses manure fertilization instead of the classic mineral nitrogen fertilization used in the standard environments of CyclesGym. Manure fertilization is characterized by both organic C and N. The rate are taken by a standard manure fertilization given by Cycles documentation.

In [5]:
from cyclesgym.envs.implementers import Fertilizer
from cyclesgym.managers import OperationManager
from pathlib import Path

class ManureFertilizer(Fertilizer):
    def __init__(self, operation_manager: OperationManager,
                 operation_fname: Path,
                 start_year: int,
                 rate: [float] = np.array([0.26, 0.0087, 0.0027])):

        nutrients = ['C_Organic', 'N_Organic', 'N_NH4']
        super(ManureFertilizer, self).__init__(operation_manager,
                                               operation_fname,
                                               nutrients,
                                               start_year)
        assert np.all(np.logical_and(0 <= rate, rate <= 1)), f'Rate must be in [0, 1]. It is {rate} instead'
        self.rate = rate

    def convert_mass(self, mass):
        #mass in Kg
        mass = mass * sum(self.rate)
        masses = {'C_Organic': mass * self.rate[0],
                  'N_Organic': mass * self.rate[1],
                  'N_NH4': mass * self.rate[2]}
        return masses

    def implement_action(self, date: datetime.date, mass: float):
        #mass in Kg
        masses = self.convert_mass(mass)
        return super(ManureFertilizer, self).implement_action(date, masses)

We can now define a new cost function for the manure fertilization. Let assume a rough cost estimate for manure to be 20$ per ton.


In [ ]:
from cyclesgym.envs.utils import date2ydoy
manure_price_dollars_per_kg = 10 * 0.001

class ManureProfitabilityRewarder(object):

    def compute_reward(self, date, delta, action=None):
        #mass in Kg
        manure_kg_per_hectare = action
        assert manure_kg_per_hectare >= 0, f'We cannot have negative fertilization'
        y, doy = date2ydoy(date)
        dollars_per_hectare = manure_kg_per_hectare * manure_price_dollars_per_kg
        return -dollars_per_hectare

The total reward is the compound of crop profitability with the cost of manure application.

In [ ]:
from cyclesgym.envs.rewarders import compound_rewarder, CropRewarder
total_reward = lambda season_manager: compound_rewarder([CropRewarder(season_manager, 'CornRM.90'), ManureProfitabilityRewarder()])

We can finally define a new environment that combines all the elements defined so far.

In [ ]:
from cyclesgym.envs.corn import Corn

class OrganicCorn(Corn):

    def _init_observer(self, *args, **kwargs):
        end_year = self.ctrl_base_manager.ctrl_dict['SIMULATION_END_YEAR']
        self.observer = trailing_and_cumulative(end_year)

    def _init_rewarder(self, *args, **kwargs):
        self.rewarder = total_reward(self.season_manager)

    def _init_implementer(self, *args, **kwargs):
        self.implementer = ManureFertilizer(operation_manager=self.op_manager,
                                            operation_fname=self.op_file,
                                            start_year=self.ctrl_base_manager.ctrl_dict['SIMULATION_START_YEAR']
                                            )

Now we can define a configuration file that is logged using wandb.

In [ ]:
import wandb
from wandb.integration.sb3 import WandbCallback
import gym
from stable_baselines3 import PPO

config = dict(start_year=1980, end_year=1980, maxN=50000, delta=7, n_actions=11,
              total_timesteps=100000, n_steps=80, batch_size=80, n_epochs=10,
              verbose=1, device='cpu', n_procs=1)

wandb.init(
    config=config,
    sync_tensorboard=True,
    project='notebook_experiments',
    monitor_gym=True,
    save_code=True,
    dir=PROJECT_PATH,
)

config = wandb.config

Now we use a subset of the configuration file to define a crop planning environment that simulate a multiyear rotation between maize and soybeans.

In [34]:
env_conf = {key: config[key] for key in ['start_year', 'end_year', 'maxN', 'delta', 'n_actions']}

def make_env():
    def f():
        env = OrganicCorn(**env_conf)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        return env
    return f

env = SubprocVecEnv([make_env() for _ in range(config['n_procs'])], start_method='fork')
env = VecMonitor(env)
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=5000., clip_reward=5000.)

We can now define the learning agent. Here we use the PPO model from the stable-baselines3 library fro simplicity.

In [35]:
model = PPO('MlpPolicy', env, n_steps=config['n_steps'], batch_size=config['batch_size'],
            n_epochs=config['n_epochs'], verbose=config['verbose'], tensorboard_log=wandb.run.dir,
            device=config['device'])

Using cpu device


Now we train the model for a number ot total step specified in the config dictionary.

In [ ]:
model.learn(total_timesteps=config["total_timesteps"], callback=[WandbCallback()])
wandb.finish()

wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`
wandb: WARNING Found log directory outside of given root_logdir, dropping given root_logdir for event file in /home/luca/Projects/plant_science/cyclesgym/wandb/run-20220616_123611-4o6iylmf/files/PPO_1



Simulation 2022_06_16_12_36_21-e398fa14-6fc7-40dc-b609-2ba2dc299658/control running ...

Simulation time: 0 seconds.
Logging to /home/luca/Projects/plant_science/cyclesgym/wandb/run-20220616_123611-4o6iylmf/files/PPO_1



ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]


Simulation 2022_06_16_12_36_24-b7d2916a-7a8f-4809-911b-131e3f71fa02/control running ...

Simulation time: 0 seconds.
-----------------------------------
| rollout/           |            |
|    ep_len_mean     | 53         |
|    ep_rew_mean     | -12188.112 |
| time/              |            |
|    fps             | 17         |
|    iterations      | 1          |
|    time_elapsed    | 4          |
|    total_timesteps | 80         |
-----------------------------------

Simulation 2022_06_16_12_36_27-85cc10a3-f4c9-4721-b7fa-b1107a25a724/control running ...

Simulation time: 0 seconds.



ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [micrCNRatio is large > 1000]
ComputeSoilCarbonBalanceMB [mi

...

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 53        |
|    ep_rew_mean          | 699.54443 |
| time/                   |           |
|    fps                  | 42        |
|    iterations           | 694       |
|    time_elapsed         | 1293      |
|    total_timesteps      | 55520     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.00117  |
|    explained_variance   | 0.936     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.000849  |
|    n_updates            | 6930      |
|    policy_gradient_loss | 1.48e-08  |
|    value_loss           | 0.0017    |
---------------------------------------

Simulation 2022_06_16_12_57_54-8e822fbd-fc47-4965-9533-b878c7687191/control running ...

Simulation time: 0 seconds.

Simulation 2022_06_16_12_57_55-8856f6fb-d


Simulation 2022_06_16_12_58_05-5da45518-d98f-43d0-af49-3d8a098b8ff4/control running ...

Simulation time: 0 seconds.

Simulation 2022_06_16_12_58_06-ca1eb382-3024-469a-a307-a18256a47237/control running ...

Simulation time: 0 seconds.
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 53        |
|    ep_rew_mean          | 699.54443 |
| time/                   |           |
|    fps                  | 43        |
|    iterations           | 703       |
|    time_elapsed         | 1305      |
|    total_timesteps      | 56240     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.00125  |
|    explained_variance   | 0.947     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.000464  |
|    n_updates            | 7020      |
|    policy_gradient_loss | -9.69e-10 |
|   


Simulation 2022_06_16_12_58_18-994133f1-add9-41ff-8081-7c33401dc845/control running ...

Simulation time: 0 seconds.
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 53        |
|    ep_rew_mean          | 699.54443 |
| time/                   |           |
|    fps                  | 43        |
|    iterations           | 711       |
|    time_elapsed         | 1317      |
|    total_timesteps      | 56880     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.00123  |
|    explained_variance   | 0.894     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.00112   |
|    n_updates            | 7100      |
|    policy_gradient_loss | -2.31e-08 |
|    value_loss           | 0.00244   |
---------------------------------------

Simulation 2022_06_16_12_58_19-b0a46721-2

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 53        |
|    ep_rew_mean          | 699.54443 |
| time/                   |           |
|    fps                  | 43        |
|    iterations           | 719       |
|    time_elapsed         | 1328      |
|    total_timesteps      | 57520     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.0012   |
|    explained_variance   | 0.928     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.000807  |
|    n_updates            | 7180      |
|    policy_gradient_loss | -1.73e-08 |
|    value_loss           | 0.00162   |
---------------------------------------

Simulation 2022_06_16_12_58_30-27fe297a-a604-4738-b320-732410fb4047/control running ...

Simulation time: 0 seconds.
---------------------------------------
| 


Simulation 2022_06_16_12_58_41-5989f91c-e171-45ac-9231-6af355435456/control running ...

Simulation time: 0 seconds.
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 53        |
|    ep_rew_mean          | 699.54443 |
| time/                   |           |
|    fps                  | 43        |
|    iterations           | 728       |
|    time_elapsed         | 1341      |
|    total_timesteps      | 58240     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.00133  |
|    explained_variance   | 0.962     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.000405  |
|    n_updates            | 7270      |
|    policy_gradient_loss | -9.16e-09 |
|    value_loss           | 0.000812  |
---------------------------------------

Simulation 2022_06_16_12_58_42-8a060533-a